`Hotel Room`

raj branch 

Here the data is belong to only one single hotel in the expedia database provided in the kaggle competation and the hotel is situated in USA Since different countries have different conventions regarding displaying taxes and fees and the value may be per night or for the whole stay. And we know that price displayed to US visitors is always per night and without taxes

Features are date_time, price_usd, srch_booking_window, srch_saturday_night_bool
and Features selected for the Auto Encoder is price_usd, srch_booking_window, srch_saturday_night_bool : 0 means not saturday night and 1 means saturday night
But Autoencoder performs better if the number of features are more . RMSE is calculated to evaluate the model in general for anomaly RMSE will be more chek 1

In [1]:
import pandas as pd
from tensorflow.keras.utils import get_file

expedia = pd.read_csv('Data_filter.csv',sep = '\t')
df = expedia
print("Read {} rows.".format(len(df)))
# df = df.sample(frac=0.1, replace=False) # Uncomment this line to sample only 10% of the dataset
df.dropna(inplace=True,axis=1) # For now, just drop NA's (rows with missing values)

df[0:5]

Read 3049 rows.


,Unnamed: 0,date_time,price_usd,srch_booking_window,srch_saturday_night_bool
0,2041,2013-03-14 11:27:28,206.0,99,1
1,3152,2013-01-03 20:48:24,186.0,6,0
2,5166,2013-01-19 16:51:27,61.0,1,0
3,10135,2013-01-26 11:34:23,72.0,116,0
4,10560,2013-04-29 09:39:50,246.0,245,0


In [ ]:
# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd
    
# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = f"{name}-{x}"
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)

In [ ]:
# Now encode the feature vector 

encode_numeric_zscore(df, 'price_usd')

encode_numeric_zscore(df, 'srch_booking_window')


In [ ]:
# Anomaly interval mean - 3*standard deviation and mean + 3*standard deviation
lower_boundry = (df['price_usd'].mean() - (3*df['price_usd'].std()))
upper_boundry = (df['price_usd'].mean() + (3*df['price_usd'].std()))


In [5]:
print(f"lower_boundry: {lower_boundry}") 
print(f"upper_boundry: {upper_boundry}")


lower_boundry: -3.000000000000004
upper_boundry: 3.000000000000004


In [ ]:
# Normal are the data which are inside the lower and upper boundary 
normal_mask = (lower_boundry <= df['price_usd']) & ( df['price_usd'] <= upper_boundry)
# Anamoly are the data which are outside the  lower and upper boundary 
Anamoly_mask = (lower_boundry > df['price_usd'] )| (df['price_usd'] > upper_boundry)

In [ ]:
df_normal = df[normal_mask]
df_Anamoly = df[Anamoly_mask]

In [8]:
print(f"Len of Normal: {len(df_normal)}") 
print(f"Len of Anamoly : {len(df_Anamoly)}")

Len of Normal: 3047
Len of Anamoly : 2


In [ ]:
# This is the numeric feature vector, as it goes to the neural net
x_normal = df_normal.iloc[:,2::].values
x_Anamoly = df_Anamoly.iloc[:,2::].values

In [10]:
x_Anamoly

array([[ 3.73155041, -0.80334557,  0.        ],
       [48.25673107, -0.52171811,  0.        ]])

In [ ]:
from sklearn.model_selection import train_test_split

x_normal_train, x_normal_test = train_test_split(
    x_normal, test_size=0.25, random_state=42)

In [12]:
print(f"Normal train count: {len(x_normal_train)}")
print(f"Normal test count: {len(x_normal_test)}")

Normal train count: 2285
Normal test count: 762


Auto Encoder

In [13]:
from sklearn import metrics
import numpy as np
import pandas as pd
from IPython.display import display, HTML 
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation

model = Sequential()
model.add(Dense(3, input_dim=x_normal.shape[1], activation='relu'))
model.add(Dense(2, activation='relu'))
model.add(Dense(3, activation='relu'))
model.add(Dense(x_normal.shape[1])) # Multiple output neurons
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x_normal_train,x_normal_train,verbose=1,epochs=100)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Train on 2285 samples
Epoch 1/100
2285/2285 [==============================] - 5s 2ms/sample - loss: 0.6396
Epoch 2/100
2285/2285 [==============================] - 0s 88us/sample - loss: 0.5710
Epoch 3/100
2285/2285 [==============================] - 0s 99us/sample - loss: 0.5331
Epoch 4/100
2285/2285 [==============================] - 0s 91us/sample - loss: 0.5021
Epoch 5/100
2285/2285 [==============================] - 0s 101us/sample - loss: 0.4755
Epoch 6/100
2285/2285 [==============================] - 0s 96us/sample - loss: 0.4531
Epoch 7/100
2285/2285 [==============================] - 0s 97us/sample - loss: 0.4329
Epoch 8/100
2285/2285 [==============================] - 0s 99us/sample - loss: 0.4136
Epoch 9/100
2285/2285 [==============================] - 0s 95us/sample - loss: 0.3962
Epoch 10/100
2285/2285 [==============================] - 0s 106us/sample - loss: 0.3815
Epoch 11/100
2285/2285 [=

In [14]:
pred = model.predict(x_normal_train)
score0 = np.sqrt(metrics.mean_squared_error(pred,x_normal_train))
pred = model.predict(x_normal_test)
score1 = np.sqrt(metrics.mean_squared_error(pred,x_normal_test))
pred = model.predict(x_normal)
score2 = np.sqrt(metrics.mean_squared_error(pred,x_normal))
pred = model.predict(x_Anamoly)
score3 = np.sqrt(metrics.mean_squared_error(pred,x_Anamoly))
print(f"Train Normal Score (RMSE): {score0}".format(score0))
print(f"Test Normal Score (RMSE): {score1}".format(score1))
print(f"Total Sample Normal Score (RMSE): {score2}")
print(f"Anomaly Sample Score (RMSE): {score3}")

Train Normal Score (RMSE): 0.21547695928103805
Test Normal Score (RMSE): 0.21317255029592558
Total Sample Normal Score (RMSE): 0.21490298506551248
Anomaly Sample Score (RMSE): 19.52843919544663
